<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#学習-vs-テストデータセットの類似性" data-toc-modified-id="学習-vs-テストデータセットの類似性-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>学習 vs テストデータセットの類似性</a></span><ul class="toc-item"><li><span><a href="#blastの実行（train-vs-test）" data-toc-modified-id="blastの実行（train-vs-test）-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>blastの実行（train vs test）</a></span></li><li><span><a href="#【出力】訓練・テストデータセット間の配列類似性" data-toc-modified-id="【出力】訓練・テストデータセット間の配列類似性-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>【出力】訓練・テストデータセット間の配列類似性</a></span></li><li><span><a href="#箱ヒゲ図" data-toc-modified-id="箱ヒゲ図-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>箱ヒゲ図</a></span></li><li><span><a href="#BLASTによる予測：AUROC,-PR-AUC" data-toc-modified-id="BLASTによる予測：AUROC,-PR-AUC-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>BLASTによる予測：AUROC, PR-AUC</a></span></li><li><span><a href="#【注：virus_IDごとになっていない】【出力】virus_IDごとの予測結果" data-toc-modified-id="【注：virus_IDごとになっていない】【出力】virus_IDごとの予測結果-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>【注：virus_IDごとになっていない】【出力】virus_IDごとの予測結果</a></span></li><li><span><a href="#【出力】ウイルス科ごとのAUROC,-PR-AUC" data-toc-modified-id="【出力】ウイルス科ごとのAUROC,-PR-AUC-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>【出力】ウイルス科ごとのAUROC, PR-AUC</a></span></li></ul></li></ul></div>

In [ ]:
#!/usr/bin/env python
import sys, re
argvs = sys.argv
sys.setrecursionlimit(10000000)

import os
import collections
import numpy as np
import pandas as pd
import array
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

import glob

## visualize
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

plt.rcParams['font.family'] = "Arial"

# sequence similairty between training and test datasets

In [32]:
virus="Coronaviridae"

In [33]:
dataset_f = "input_examples/known_virus_final_dataset_Coronaviridae.txt"

df_curated = pd.read_csv(dataset_f, index_col=0)
df_curated.head(3)

,refseq.id,fold_0,label,sampling,database,date,fold_1,fold_2,fold_3,fold_4,host_label,Genus,virus_ID,Family,Species
73331,OQ175110,train,0,normal,GenBank,20240206,pred,train,train,train,0.0,NaN,OQ175110,Coronaviridae,Bat Coronavirus MrGD17
73332,OQ175111,train,0,normal,GenBank,20240206,pred,train,dev,train,0.0,NaN,OQ175111,Coronaviridae,Bat Coronavirus MrGD17
73333,OQ175112,train,0,normal,GenBank,20240206,pred,train,train,dev,0.0,NaN,OQ175112,Coronaviridae,Bat Coronavirus MrGD17


In [34]:
df_curated.groupby(["Family", "sampling", "label"]).agg(count=("virus_ID", "nunique"))

count
Family        sampling label       
Coronaviridae normal   0       2154
                       1        529

## preparing fasta sequences

In [35]:
virus="Coronaviridae"
output_dir = "output_examples"

fold_l = ["fold_0", "fold_1", "fold_2", "fold_3", "fold_4"]

## test for fold_0 dataset
for fold in fold_l[:1]:
    df_train_human = df_curated[(df_curated[fold] == "train") & (df_curated["label"] == 1)]
    df_train_animal = df_curated[(df_curated[fold] == "train") & (df_curated["label"] == 0)]
    
    df_pred = df_curated[df_curated[fold] == "pred"]

    output_train_human_f = "/".join(map(str, [output_dir, "train_human_acc_" + fold + ".txt"]))
    output_train_animal_f =  "/".join(map(str, [output_dir, "train_animal_acc_" + fold + ".txt"]))

    output_pred_f = "/".join(map(str, [output_dir, "pred_" + fold + ".txt"]))

    for output_f_1, df_o in zip([output_train_human_f, output_train_animal_f, output_pred_f], 
                                [df_train_human, df_train_animal, df_pred]):
        acc_l = df_o["refseq.id"].values.tolist()

        f = open(output_f_1, "w")
        f.write("\n".join(map(str, acc_l)))
        f.close()

## blastn

In [37]:
%%bash

virus="Coronaviridae"
fold="fold_0"

output_dir="output_examples"

train_human_f=${output_dir}/train_human_acc_${fold}.txt
train_animal_f=${output_dir}/train_animal_acc_${fold}.txt

test_acc_f=${output_dir}/pred_${fold}.txt

## extract sequences
cat ../002_Data_splitting/input_examples/${virus}.curated.fasta | \
    seqkit grep -f ${train_human_f} \
    > ${output_dir}/train_human_acc_${fold}.fasta
    
cat ../002_Data_splitting/input_examples/${virus}.curated.fasta | \
    seqkit grep -f ${train_animal_f} \
    > ${output_dir}/train_animal_acc_${fold}.fasta
    
cat ../002_Data_splitting/input_examples/${virus}.curated.fasta | \
    seqkit grep -f ${test_acc_f} \
    > ${output_dir}/pred_acc_${fold}.fasta

### performe blastn   
makeblastdb \
    -in ${output_dir}/train_human_acc_${fold}.fasta \
    -out ${output_dir}/train_human_acc_${fold}.DB \
    -dbtype nucl

blastn \
    -query ${output_dir}/pred_acc_${fold}.fasta \
    -db ${output_dir}/train_human_acc_${fold}.DB \
    -out  ${output_dir}/test_human_acc_${fold}.blastn \
    -outfmt '6 qseqid sseqid evalue bitscore length qlen slen pident' \
    -max_target_seqs 5 \
    -evalue 1e-4

###
makeblastdb \
    -in ${output_dir}/train_animal_acc_${fold}.fasta \
    -out ${output_dir}/train_animal_acc_${fold}.DB \
    -dbtype nucl

blastn \
    -query ${output_dir}/pred_acc_${fold}.fasta\
    -db ${output_dir}/train_animal_acc_${fold}.DB \
    -out  ${output_dir}/test_animal_acc_${fold}.blastn \
    -outfmt '6 qseqid sseqid evalue bitscore length qlen slen pident' \
    -max_target_seqs 5 \
    -evalue 1e-4
    
cat \
    ${output_dir}/test_human_acc_${fold}.blastn \
    ${output_dir}/test_animal_acc_${fold}.blastn \
    > ${output_dir}/pred_${fold}.blastn



Building a new DB, current time: 05/01/2024 14:00:33
New DB name:   /Users/junna/Desktop/research/_2024/001_VirusBERT/000_git-hub/BERT-infect_2024/003_Data_predictability/output_examples/train_human_acc_fold_0.DB
New DB title:  output_examples/train_human_acc_fold_0.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/junna/Desktop/research/_2024/001_VirusBERT/000_git-hub/BERT-infect_2024/003_Data_predictability/output_examples/train_human_acc_fold_0.DB
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 324 sequences in 0.089149 seconds.




Building a new DB, current time: 05/01/2024 14:00:49
New DB name:   /Users/junna/Desktop/research/_2024/001_VirusBERT/000_git-hub/BERT-infect_2024/003_Data_predictability/output_examples/train_animal_acc_fold_0.DB
New DB title:  output_examples/train_animal_acc_fold_0.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Users/junna/Desktop/research/_2024/

[INFO] 324 patterns loaded from file
[INFO] 1352 patterns loaded from file
[INFO] 590 patterns loaded from file


## Predicted results by BLASTn

In [38]:
blast_l = []

for fold in fold_l[:1]:
    blast_f = "/".join(map(str, [output_dir, "pred_" + fold + ".blastn"]))
    
    if os.path.exists(blast_f):
        df_blast = pd.read_csv(blast_f, sep="\t", 
                                names = ["query", "subject", "evalue", "bitscore", "alen", "qlen", "slen", "pident"])
        
        df_blast["coverage"] = df_blast["alen"] / df_blast["qlen"] * 100
        
        # unpredictable: the aligned length of the top hit sequence did not cover >50% of the query sequence
        df_blast.loc[df_blast["coverage"] < 50, "pident"] = 0
        
        df_blast_tophit = df_blast.groupby("query", group_keys=False).apply(lambda x: x[x["bitscore"] == x["bitscore"].max()])
        
        blast_l.append(df_blast_tophit)
    else:
        print (blast_f)

In [39]:
df_blast_result = pd.concat(blast_l)
df_blast_result.head()

,query,subject,evalue,bitscore,alen,qlen,slen,pident,coverage
8601,KU893862,KM392224,0.0,51755.0,28032,28032,28032,99.989,100.000000
8606,KU893863,KM392224,0.0,51703.0,28031,28031,28032,99.954,100.000000
8611,KU893864,KM392224,0.0,51727.0,28031,28039,28032,99.968,99.971468
8616,KU893865,KM392224,0.0,51722.0,28029,28029,28032,99.971,100.000000
8621,KU893866,KM392224,0.0,51716.0,28032,28043,28032,99.968,99.960775


## add infectivity label based on the tophit sequence

In [40]:
df_subject = df_curated.loc[:, ["refseq.id", "label"]]
df_subject = df_subject.rename(columns={"refseq.id": "subject", "label": "subject_label"})
df_subject = df_subject.drop_duplicates()

df_blast_result_1 = df_blast_result.merge(df_subject, on="subject", how="left")
df_blast_result_1.head(3)

,query,subject,evalue,bitscore,alen,qlen,slen,pident,coverage,subject_label
0,KU893862,KM392224,0.0,51755.0,28032,28032,28032,99.989,100.000000,0
1,KU893863,KM392224,0.0,51703.0,28031,28031,28032,99.954,100.000000,0
2,KU893864,KM392224,0.0,51727.0,28031,28039,28032,99.968,99.971468,0


In [41]:
## 
df_blast_result_2 = df_curated.merge(df_blast_result_1, left_on = ["refseq.id"], 
                                        right_on=["query"])

In [42]:
df_blast_result_2[(df_blast_result_2["label"].isnull()) |
                 (df_blast_result_2["subject_label"].isnull()) | 
                 (df_blast_result_2["pident"].isnull())]

,refseq.id,fold_0,label,sampling,database,date,fold_1,fold_2,fold_3,fold_4,host_label,Genus,virus_ID,Family,Species,query,subject,evalue,bitscore,alen,qlen,slen,pident,coverage,subject_label


## unpredictable cases

In [43]:
##
df_blast_result_2 = df_blast_result_2.astype({"label":int, "subject_label":int, "pident":float})

## unpredictable: showed both human- and animal-infecting viruses with the same bitscore
df_g = df_blast_result_2.groupby("query").agg(count=("subject_label", "nunique"))
acc_l = df_g[df_g["count"] > 1].index.tolist()

df_blast_result_2.loc[(df_blast_result_2["query"].isin(acc_l)) & (df_blast_result_2["label"] == 0), "subject_label"] = 1
df_blast_result_2.loc[(df_blast_result_2["query"].isin(acc_l)) & (df_blast_result_2["label"] == 1), "subject_label"] = 0

## unpredictable: there was no hit sequence with an E-value of >1e-4
df_blast_result_2.loc[(df_blast_result_2["query"].isnull()) & (df_blast_result_2["label"] == 0), "subject_label"] = 1
df_blast_result_2.loc[(df_blast_result_2["query"].isnull()) & (df_blast_result_2["label"] == 1), "subject_label"] = 0

df_blast_result_2.loc[(df_blast_result_2["pident"] == 0) & (df_blast_result_2["label"] == 0), "subject_label"] = 1
df_blast_result_2.loc[(df_blast_result_2["pident"] == 0) & (df_blast_result_2["label"] == 1), "subject_label"] = 0

In [44]:
df_blast_result_2.head(2)

,refseq.id,fold_0,label,sampling,database,date,fold_1,fold_2,fold_3,fold_4,host_label,Genus,virus_ID,Family,Species,query,subject,evalue,bitscore,alen,qlen,slen,pident,coverage,subject_label
0,ON075821,pred,0,normal,GenBank,20240206,dev,train,dev,train,0.0,Alphacoronavirus,ON075821,Coronaviridae,Porcine epidemic diarrhea virus,ON075821,KJ584361,0.0,50386.0,28066,28063,28038,99.091,100.01069,0
1,OR230676,pred,0,normal,GenBank,20240206,train,train,train,train,0.0,Deltacoronavirus,OR230676,Coronaviridae,Coronavirus HKU15,OR230676,MF280390,0.0,46737.0,25402,25420,25402,99.878,99.92919,0


## AUROC, PR-AUC scores

In [45]:
from sklearn.metrics import roc_auc_score, average_precision_score

df_result = df_blast_result_2

metrics_d = {"model":[], "virus_family":[], "auroc":[], "pr-auc":[]}

result_l = []

model = "blast"

df_1 = df_result.groupby("virus_ID").agg(average_prob=("subject_label", "mean"),
                                    label=("label", "max"))

y_test = df_1["label"]
y_pred_prob = df_1["average_prob"]

if len(set(y_test)) == 1:
    auc = 0
    average_precision = 0
else:
    auc = roc_auc_score(y_test, y_pred_prob)
    average_precision = average_precision_score(y_test, y_pred_prob)

metrics_d["model"].append(model)
metrics_d["virus_family"].append(virus)
metrics_d["auroc"].append(auc)
metrics_d["pr-auc"].append(average_precision)

df_metrics = pd.DataFrame(metrics_d)
df_metrics.sort_values("virus_family")

,model,virus_family,auroc,pr-auc
0,blast,Coronaviridae,0.945151,0.759885


## outputs

In [46]:
output_f = "output_examples/blast_prediction_results.csv"

df_blast_result_2.to_csv(output_f, index=False)

In [47]:
output_f = "output_examples/blast_metrics.csv"

df_metrics.to_csv(output_f, index=False)